<a href="https://colab.research.google.com/github/sandeep-srinivasan/Data-Mining-Project/blob/master/Semester_project_part2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os
import re
from collections import OrderedDict
import tensorflow as tf

In [0]:
# The following class and two functions have been taken from wikipedia at https://en.wikipedia.org/wiki/Trie#Algorithms

class Node():
    def __init__(self):
       # Note that using dictionary for children (as in this implementation) would not allow lexicographic sorting mentioned in the next section (Sorting),
       # because ordinary dictionary would not preserve the order of the keys
        self.children = {}  # mapping from character ==> Node
        self.value = None

def find(node, key):
    for char in key:
        if char in node.children:
            node = node.children[char]
        else:
            return None
    return node.value
    
def insert(root, string, value):
    node = root
    index_last_char = None
    for index_char, char in enumerate(string):
        if char in node.children:
            node = node.children[char]
        else:
            index_last_char = index_char
            break

    # append new nodes for the remaining characters, if any
    if index_last_char is not None: 
        for char in string[index_last_char:]:
            node.children[char] = Node()
            node = node.children[char]

    # store value in the terminal node
    node.value = value

# The following two functions have been written by the programmers for additional purposes of the trie    
    
def find_multiple(node, keys):
    # Return values for multiple Keys in the trie Node in order that keys are presented
    holder = node
    vals = [None]*len(keys)
    counter = 0
    for key in keys:
        node = holder
        for char in key:
            if char in node.children:
                node = node.children[char]
        vals[counter] = node.value
        counter += 1
    return vals

def update(node, key, difference):
    # Change the value which is currently stored for the Key in the trie Node by a value of Difference
    for char in key:
        if char in node.children:
            node = node.children[char]
    node.value += difference  

In [6]:
# the sets are a data structure which was solely used for checking results with the trie
listtopics=set()
listplaces=set()
listwords = set()
article_tries = [None]*21812
# counts for no topics and no places in articles
cntnotop=0 
cntnoplc=0 
# Trie for the different topics, locations, and count of every word present across all articles
trieTopics = Node()
trieLoc = Node()
WordCount = Node()
# csv to hold all word values per article (row)

articleCount = 0

for i in range(0,22):
    # over all files
    if(i>=10):
        # file names differ by the #, which is double digit for i>=10
        filename = 'reut2-0'+str(i)+'.sgm'
    else:
        filename = 'reut2-00'+str(i)+'.sgm'
    path = ''+filename
    file = open(path, 'rb')
    data = file.read()
    x = re.findall(r'<REUTERS(.*?)</REUTERS>', data.decode("windows-1252"), re.DOTALL)
    # finds all instances of "<REUTERS . . ." in a given file and save them 

    for j in range(0,len(x)):
        # for all articles in a file since every article starts with the REUTERS tag
        yTopic = re.findall(r'<TOPICS>(.*?)</TOPICS>', x[j], re.DOTALL)
        # store all topics in an article since an article can have multiple topics
        for k in range(0,len(yTopic)):
            lt = yTopic[k]

            article_topics = Node()
            topics = re.findall(r'<D>(.*?)</D>', lt, re.DOTALL)
            # Make sure D tag does not included as part of the topic name
            if(len(topics)==0):
                # length is 0 when there is no topic
                cntnotop=cntnotop+1
            for l in topics:
                # for every topic found in an article
                if (find(trieTopics,l) == None):
                    # check if the topic is already in the trie, and if not insert it with value 1
                    insert(trieTopics, l, 1)
                    insert(article_topics, l, 1)
                elif (find(article_topics, l) == None):
                    insert(article_topics, l, 1)
                    update(trieTopics, l, 1)
                else:
                    # its been found already in the trie so increase the value by 1
                    update(trieTopics, l, 1)
                    update(article_topics, l, 1)
                #article_topics.append(l)
                listtopics.add(l)
        
        article_places = []
        yPlace = re.findall(r'<PLACES>(.*?)</PLACES>', x[j], re.DOTALL)
        for k in range(0,len(yPlace)):
            lt = yPlace[k]
            places = re.findall(r'<D>(.*?)</D>', lt, re.DOTALL)
            if(len(places)==0):
                cntnoplc=cntnoplc+1
            for l in places:
                if (find(trieLoc, l) == None):
                    insert(trieLoc, l, 1)
                else:
                    update(trieLoc, l, 1)
                article_places.append(l)
                listplaces.add(l)

        article_words = Node()        
        yBody = re.findall(r'<BODY>(.*?)</BODY>', x[j], re.DOTALL)
        for b,word in enumerate(yBody):
            # split the body into a bunch of different words
            body = word.split()
            body = [element.lower() for element in body] ; body            
            for l in body:
                if (find(WordCount, l) == None):
                    insert(WordCount, l, 1)
                    insert(article_words, l, 1)
                elif (find(article_words, l) == None):
                    insert(article_words, l, 1)
                    update(WordCount, l, 1)
                else:
                    update(WordCount, l, 1)
                    update(article_words, l, 1)
                listwords.add(l)
        #print (article_topics)
        article_tries[articleCount] = [article_topics, article_places, article_words]
        articleCount += 1

        
# end of main for loop for all files

# Print statements for the distinct list of topics, distinct list of places, and counts of topic-less and/or place-less 
# articles.  Although, we used set data structures to display the different keys here, it is easy to fetch values for keys 
# using a trie displayed below each.  Usage of sets was only done as part of "developing our domain-specific knowledge".
listtopics = list(listtopics)
listplaces = list(listplaces)
listwords = list(listwords)
#print(listtopics)
print(find_multiple(trieTopics, listtopics))
#print(listplaces)
#print(find_multiple(trieLoc, listplaces))
#print("Data objects with no entries for topics: " + str(cntnotop))
#print("Data objects with no entries for places: " + str(cntnoplc))

[44, 51, 8, 2, 130, 1, 628, 4, 7, 78, 8, 1, 65, 63, 43, 2, 6, 76, 9, 35, 14, 145, 163, 3, 1, 4, 2, 28, 15, 51, 1, 1, 29, 137, 306, 65, 6, 3, 192, 67, 4, 1, 3, 184, 1, 1, 17, 15, 1, 1, 552, 1, 25, 2448, 10, 32, 4, 6, 3, 513, 33, 3, 305, 4, 69, 217, 17, 3, 35, 1, 84, 3, 25, 7, 254, 116, 3, 4, 27, 8, 27, 7, 3987, 3, 120, 190, 76, 17, 11, 21, 114, 12, 112, 32, 35, 37, 634, 2, 1, 2, 2, 21, 41, 27, 7, 75, 67, 2, 1, 3, 801, 3, 135, 2, 54, 63, 1, 18, 3, 2]


In [7]:
# Some tests for "finds" on the tries are shown below

print (find(trieLoc, "usa"))

12542


In [8]:
print(find(trieTopics, "sugar"))

184


In [9]:
print (find_multiple(trieLoc, ['usa', 'west-germany']))

[12542, 567]


In [10]:
print (find(WordCount, 'agriculture'))
print (find(WordCount, 'a'))

849
50596


In [11]:
print (len(listtopics) ) #120
trieVals = find_multiple(trieTopics, listtopics)
#print (trieVals)
#topictrie[0] = listtopics
#topictrie[1] = trieVals
#print (topictrie[1])
'''with open('output_trie_topics.csv', 'w') as csvfile:
    fieldnames = ['topic', 'value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(listtopics)):
        writer.writerow({'topic': topictrie[0][i], 'value': topictrie[1][i]})'''
#np.savetxt("output_trei_topics.csv", topictrie, delimiter=",")
print (len(find_multiple(article_tries[0][2], listwords)))

120
104121


In [12]:
'''
print (len(article_tries))
print (article_tries[1][0])
with open('output_article_words.csv', 'w') as csvfile:
    #fieldnames = ['article#', 'topics', 'places', listwords]
    #writer = csv.writer(csvfile)
    #writer.writeheader()
    for i in range(len(article_tries)):
        topicString = ''
        placeString = ''
        valueString = ''
        wordVals = find_multiple(article_tries[i][2], listwords)
        if article_tries[i] != None:
            for string in article_tries[i][0]:
                topicString +=  string + ' '
            for string in article_tries[i][1]:
                placeString +=  string + ' '
            for value in wordVals:
                valueString +=  str(value) + ', '
        csvfile.write(str(i) + ',' + topicString + ', ' + placeString + ', ' + valueString + '\n')
    '''

"\nprint (len(article_tries))\nprint (article_tries[1][0])\nwith open('output_article_words.csv', 'w') as csvfile:\n    #fieldnames = ['article#', 'topics', 'places', listwords]\n    #writer = csv.writer(csvfile)\n    #writer.writeheader()\n    for i in range(len(article_tries)):\n        topicString = ''\n        placeString = ''\n        valueString = ''\n        wordVals = find_multiple(article_tries[i][2], listwords)\n        if article_tries[i] != None:\n            for string in article_tries[i][0]:\n                topicString +=  string + ' '\n            for string in article_tries[i][1]:\n                placeString +=  string + ' '\n            for value in wordVals:\n                valueString +=  str(value) + ', '\n        csvfile.write(str(i) + ',' + topicString + ', ' + placeString + ', ' + valueString + '\n')\n    "

In [0]:
def weight_variable(shape, myname):
    initial = tf.truncated_normal(shape, stddev=0.1, name=myname)
    return tf.Variable(initial)


def bias_variable(shape, myname):
    initial = tf.constant(0.1, shape=shape, name=myname)
    return tf.Variable(initial)

In [0]:
# NETWORK

# 1st layer
x = tf.placeholder(tf.float32, shape=[None, 104121], name="x")
y_input = tf.placeholder(tf.float32, shape=[None, 120], name="y_input")

W = weight_variable([104121, 100], "W")
b = bias_variable([100], "b")
y_1 = tf.nn.leaky_relu(tf.add(tf.matmul(x, W), b))

W2 = weight_variable([100, 120], "W2")
b2 = bias_variable([120], "b2")
y_test = tf.add(tf.matmul(y_1, W2), b2, name="out")

In [0]:
ssd = tf.squared_difference(y_input, y_test)
test = tf.squeeze(ssd)
#divres = tf.div(test, ugh2)
cross_entropy = tf.reduce_mean(ssd)  # switched ssd in for divres
tv = tf.trainable_variables()
grads = list(zip(tf.gradients(cross_entropy, tv), tv))
for grad, variables in grads:
    variable_3 = grad
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cross_entropy, name="gradDescent")
# switched adam to gradientdescent and removed epsilon of e-3

myind = tf.argmax(y_test,1)
myindTrue = tf.argmax(y_input,1)
myacc = tf.equal(myind,myindTrue)

some = tf.clip_by_value(ssd, clip_value_min=1e-7, clip_value_max=1e6)  # switched ssd in for divres
condition = tf.less(some, 1e-5)  # changed 1e-5 to 1
correct_prediction = tf.where(condition, tf.ones_like(condition), tf.zeros_like(condition))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="acc")
tf.summary.scalar('acc', accuracy)
merge = tf.summary.merge_all()
saver = tf.train.Saver()
tf.add_to_collection("optimizer", train_step)

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # train_writer = tf.summary.FileWriter('beginner', sess.graph)
    train_writer2 = tf.summary.FileWriter('validation', sess.graph)
    # sess.run()
    sess.run(tf.global_variables_initializer())
    # example input: find_multiple(article_tries[0][2], listwords)
    # input setup
    train_size = 5  # 1234
    test_size = 1
    batch_size = 10
    i = 0
    yans = [None] * train_size
    xans = [None] * train_size
    xans2 = [None] * test_size  # 26
    yans2 = [None] * test_size
    epochs = 0
    count = 0
    count2 = 0
    loss = 1e10
    while i < 6:  
        temp = []
        temp2 = []
        mytemp = find_multiple(article_tries[i][2], listwords)
        mytemp = [j if (j != None) else 0 for j in mytemp]
        mytemp2 = find_multiple(article_tries[i][0], listtopics)
        mytemp2 = [j if (j != None) else 0 for j in mytemp2]
        #print (mytemp)
        for j in range(len(mytemp)):
            temp.append(float(mytemp[j]))
        for j in range (len(mytemp2)):
            temp2.append(float(mytemp2[j]))
        if i % 6  != 0:
            xans[count] = temp
            yans[count] = temp2
            count += 1
        else:
            xans2[count2] = temp
            yans2[count2] = temp2
            count2 += 1
        i +=1
    yans = np.array(yans, dtype=np.float64)
    yans2 = np.array(yans2, dtype=np.float64)
   # yans2[0] = np.array([i for i in range(120)], dtype=np.float)
    print (yans2.shape)
    while epochs < 1000000:
            [summary2, acc, res] = sess.run([merge, accuracy, y_test], feed_dict={x: xans2, y_input: yans2})
            train_writer2.add_summary(summary2, epochs)
            if epochs % 100 == 0:
                [summaryOut, train_accuracy, something, true_val, c, myacc2, ind, ind2, yin] = sess.run([merge, accuracy, y_test, y_input,
                                            cross_entropy, myacc, myind, myindTrue, y_input], feed_dict={x: xans2, y_input: yans2})
                if c < loss:
                  #saver.save(sess, './my_test_model', global_step=epochs)
                  #np.savetxt("output_fourier_check.csv", true_val, delimiter=",")
                  loss = c
                a = abs(something - true_val)
                numpy_accuracy = (a <= 1e-5).mean()   # changed 1e-5 to 1
                try:
                    # print (v3)
                    print('step %d, training accuracy %g' % (epochs, train_accuracy))
                    print('step %d, training accuracy %g' % (epochs, numpy_accuracy))
                    #print('step %d, idk accuracy %g' % (epochs, myacc2))
                    print('ind %d, index true %g' % (ind, ind2))
                    print('validation accuracy %g' % acc)
                    print (something)
                    print(yin)
                    print (c)
                except OSError as e:
                    pass
            train_step.run(feed_dict={x: xans, y_input: yans})
            epochs += 1
    # print (something[0])
    #[acc, res] = sess.run([accuracy, y_test], feed_dict={x: xans2, y: yans2})
    #np.savetxt("output_fourier9.csv", res, delimiter=",")
    #print('validation accuracy %g' % acc)

(1, 120)
step 0, training accuracy 0
step 0, training accuracy 0
ind 82, index true 17
validation accuracy 0
[[ 5.3865826e+01  8.4654472e+01  3.6793789e+01 -9.8859299e+01
   2.1472620e+01  1.5386524e+01 -1.0304694e+01 -1.4529668e+01
  -4.6947430e+01 -8.8395233e+00  2.8175385e+01 -5.4447510e+01
  -3.4337166e+01 -9.3949671e+00  3.8573982e+01 -1.0633196e+01
  -1.5294166e+01 -1.5532207e+01 -3.4680565e+01  3.1186535e+01
   6.2336208e+01 -2.4726213e+01 -6.2888657e+01  3.8887291e+01
   3.3421810e+01  8.5356712e+01 -3.2549126e+01  3.3659962e+01
  -1.2542417e+01 -1.9500973e+01  2.2224991e+00  1.5150831e+01
   1.5163439e+01 -2.4055624e+01 -7.1474518e+01 -4.0660271e+01
  -5.6890137e+01 -7.9445610e+01  2.1007387e+01  4.9988529e+01
   4.2466072e+01 -2.4735370e+00 -2.9143215e+01  5.3276505e+01
  -6.1582214e+01  3.0407610e+01  3.3607960e+01  2.1103662e+01
  -5.4584995e+01  8.2504662e+01 -8.0282621e+00 -4.4799152e+01
  -3.4564922e+01 -4.7794456e+01 -6.0056679e+01  4.7105808e+01
   1.4708441e+01  2.737